<a href="https://colab.research.google.com/github/archyyu/GPT-from-MLP-to-RNN-to-Transformer/blob/main/GPT_by_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import requests
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

In [63]:
# Data I/O

url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
data = response.text

chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(f'data has {data_size} characters, {vocab_size} unique.')

char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

data has 1115394 characters, 65 unique.


In [64]:
class Head(nn.Module):
  def __init__(self, input_size, sequence_length, head_size):
    super(Head, self).__init__()
    self.C = input_size
    self.L = sequence_length
    self.head_size = head_size
    self.q = nn.Linear(self.C, head_size, bias=False)
    self.k = nn.Linear(self.C, head_size, bias=False)
    self.v = nn.Linear(self.C, head_size, bias=False)

  def forward(self, x):
    q = self.q(x)
    k = self.k(x)
    v = self.v(x)

    wei = q @ k.transpose(-2, -1) * (self.head_size ** -0.5)
    tril = torch.tril(torch.ones(self.L, self.L))
    wei = wei.masked_fill(tril == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)

    out = wei @ v
    return out


In [65]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads, input_size, sequence_length, head_size):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads

    self.heads = nn.ModuleList([
        Head(input_size, sequence_length, head_size) for _ in range(num_heads)
    ])

    self.final_linear = nn.Linear(num_heads * head_size, input_size)
    self.relu = nn.ReLU()

  def forward(self, x):

    head_outputs = [head(x) for head in self.heads]
    concatenated_output = torch.cat(head_outputs, dim=-1)
    final_output = self.relu(self.final_linear(concatenated_output))

    return final_output

In [66]:
class BlockAttention(nn.Module):
  def __init__(self, num_heads, input_size, sequence_length, head_size):
    super(BlockAttention, self).__init__()
    self.multiheads = MultiHeadAttention(num_heads, input_size, sequence_length, head_size)
    self.norm = nn.LayerNorm(input_size)

  def forward(self, x):
    inter_result = x + self.multiheads(x)
    final_output = self.norm(x + inter_result)
    return final_output

In [85]:
class Decoder(nn.Module):
  def __init__(self, num_heads, vocab_size, embedding_size, sequence_length, head_size):
    super(Decoder, self).__init__()

    self.em = nn.Embedding(vocab_size, embedding_size)

    self.blocks = nn.ModuleList([BlockAttention(num_heads, embedding_size, sequence_length, head_size) for _ in range(4)])
    self.fw = nn.Linear(sequence_length * embedding_size, vocab_size, bias=False)

  def forward(self, x):
    x = self.em(x)
    for block in self.blocks:
      x = block(x)
    B,T,C = x.shape
    x = x.view(B,1,T*C)
    return self.fw(x)

In [ ]:
batch_size = 8
sequence_length = 8
input_size = 32
num_heads = 4

head_size = 12

# Example input tensor x = torch.randn(B,T,C)
x = torch.randn(batch_size, sequence_length, input_size)

# Initialize the multi Head
# head = MultiHeadAttention(num_heads, input_size, sequence_length, head_size)

head = Decoder(num_heads, input_size, sequence_length, head_size)

# Forward pass
print(x.shape)
output = head(x)

print(output.shape)

In [87]:
# Hyperparameters
hidden_size = 100
embedding_dim = 20
seq_length = 8
learning_rate = 1e-1
batch_size = 20
num_heads = 4
head_size = 12

criterion = nn.CrossEntropyLoss()
    #Decoder(num_heads, input_size, sequence_length, head_size)
model = Decoder(num_heads, vocab_size, embedding_dim, seq_length, head_size)
optimizer = optim.Adagrad(model.parameters(), lr=learning_rate)

In [88]:
def generate_mini_batch():
  # Assuming batch_size is a variable representing the desired batch size
  # and data is your input sequence data

  # Initialize lists to store input sequences and corresponding targets for the minibatch
  batch_inputs = []
  batch_targets = []

  # Loop to generate the minibatch
  for _ in range(batch_size):
    # Randomly select a starting point for the sequence
    p = np.random.randint(0, len(data) - seq_length - 1)

    # Extract a sequence of characters and convert them to indices
    inputs = torch.tensor([char_to_ix[ch] for ch in data[p:p + seq_length]], dtype=torch.long).view(1, -1)

    # Extract the target character and convert it to an index
    target = torch.tensor([char_to_ix[data[p + seq_length]]], dtype=torch.long).view(1, -1)

    # Append the input sequence and target to the minibatch lists
    batch_inputs.append(inputs)
    batch_targets.append(target)

  # Combine the lists into tensors to form the minibatch
  minibatch_inputs = torch.cat(batch_inputs, dim=0)
  minibatch_targets = torch.cat(batch_targets, dim=0)
  return minibatch_inputs, minibatch_targets

In [ ]:
# Training loop
stopi = []
lossi = []
num_iterations = 5
for iteration in range(num_iterations):

  for p in range(len(data) - seq_length):

    # inputs = torch.tensor([char_to_ix[ch] for ch in data[p:p + seq_length]], dtype=torch.long).view(1, -1)
    # targets = torch.tensor([char_to_ix[ch] for ch in data[p + seq_length]], dtype=torch.long).view(-1)

    inputs, targets = generate_mini_batch()
    optimizer.zero_grad()
    predict_char = model(inputs)

    loss = criterion(predict_char.view(-1, 65), targets.view(-1))

    loss.backward()

    for param in model.parameters():
      if param.grad is not None:
        param.grad.data.clamp_(-5, 5)

    optimizer.step()

    if p % 2000 == 0:
      print(f'Iteration {(iteration + 1) * p}, Loss: {loss.item()}')
      stopi.append((iteration + 1) * p)
      lossi.append(loss.item())

Iteration 0, Loss: 2.463075876235962


In [112]:
start = "First Citizen"

for i in range(1000):
  lll = start[-seq_length:]
  ll = torch.tensor([char_to_ix[ch] for ch in lll], dtype=torch.long).view(1, -1)
  outputs = model(ll)
  p = nn.functional.softmax(outputs, dim=-1).detach().numpy().ravel()
  ix = np.random.choice(range(vocab_size), p=p)
  ix = torch.tensor(ix, dtype=torch.long).view(1, 1)
  start += ix_to_char[ix[0][0].item()]

print(start)


First Citizends?
Thisin stath trorin therst epe!
I wel bomy shark,
Owand thatilg lidgeg, I c, Ruvin to
The's aw thy his hly mr aow the rad secents uuthes.

FOROM:
Anch
Me Himquti: Marepre, ror wrsolegheg, our Sordarerd To, ce lime,
The
The EKube sowe he dvallde
Therese,
lE wevurd haref
wy uu haves:
Gre The mere'ces beverneg,
he tobssnt bed; at noin th art's. Rnd t, beo fore,
Thais , aseswed,
Ald thing, The prat.

LUCEPIONRE:
Oou, alermla?

GLE MIO:
OTEN SeNR maig,
To mure! I lakd co tele vert werle.
As the thad ha burann fiay asstunen: the llis med; are douns the ke resthild ald
Brt come.
NPAnd:
Heran it tee.
CLANIY:
ADToos Kevefere stagr wor, hou f ansorn;
Ast y, lo ntow inchas re ofd ar hatr sige and the hy y tercar ee shice von tiell.

GRANTE:
If er caponeklatis sie the ove kead, wit beade. Inthit boow ur tot ys med bepotad se fore, ill ove goubu nest dillA ha Tere,, ghow I died; the my s y arveas, ruris
Btte foove, Thate woursnye: werellt toy brerea, an tt bo t weve theu ly in the 